In [2]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Load Data

In [3]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset
path_to_complete_dataset = "verified/complete_dataset"
path_to_tiles_folder_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"tiles")
path_positive_images_complete_dataset = os.path.join(parent_directory, path_to_complete_dataset,"chips_positive")
path_to_verified_sets = os.path.join(parent_directory, path_to_complete_dataset,"verified//verified_sets")

## Tile names and urls for all tiles apart of the dataset (complete array)

In [4]:
tile_names_tile_urls_complete_array = fc.add_formatted_and_standard_tile_names_to_tile_names_time_urls(tile_names_tile_urls_complete_array)
print(tile_names_tile_urls_complete_array.shape)

tile_names_tile_urls_complete_array_unique_standard_tile_names, tile_names_tile_urls_complete_array_unique_formatted_tile_names = fc.unique_formatted_standard_tile_names(tile_names_tile_urls_complete_array)
#Save unique standard tile names
np.save("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy", tile_names_tile_urls_complete_array_unique_standard_tile_names)
#pd.DataFrame(np.load("image_download_azure/tile_name_tile_url_complete_array.npy")).to_csv("image_download_azure/tile_name_tile_url_complete_array.csv")

(2610, 4)
unique formatted tile names (2540, 4)
unique standard tile names (2439, 4)


In [5]:
#The file paths of .jpgs in the entire storage space
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

path_storage_space = os.path.join(parent_directory)
jpgs_storage_space = glob(path_storage_space + "/**/*.jpg", recursive = True)
print(len(jpgs_storage_space))

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#The file names (with and without extensions) for all jpgs in the storage space 
#jpgs_ext, jpgs_without_ext = fc.jpg_path_to_jpg_name_formatted(jpgs_storage_space)

2293088


In [6]:
#Determine the corresponding tile names (formatted) for given paths of jpgs 
tile_names_formatted_storage_space = fc.jpg_path_to_tile_name_formatted(jpgs_storage_space)
tile_names_formatted_storage_space = np.unique(tile_names_formatted_storage_space)
tile_names_formatted_storage_space.shape

#Deteremine the tile names (standard) for given paths of jpgs 
tile_names_standard_storage_space = []
for tile_name_formatted_storage_space in tile_names_formatted_storage_space:
    tile_names_standard_storage_space.append(tile_name_formatted_storage_space.split("_",4)[4])
    
#Create a 2d array of the standard and formatted tile names of the jpgs given
tile_names_in_storage_space_standard_formatted = np.hstack((np.array(tile_names_standard_storage_space)[:,None], tile_names_formatted_storage_space[:,None]))

In [4]:
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_name_standard in tile_names_in_storage_space_standard_formatted[:,0]: #iterate over the tiles that have actually been annotated
    strings_with_substring = [string for string in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2] if tile_name_standard in string] #check if a given tile is in the complete_array 
    if len(strings_with_substring) == 0:
        counter_0 += 1
        print(tile_name_standard)
    if len(strings_with_substring) == 1:
        counter_1 += 1
    if len(strings_with_substring) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)

0 2437 0


In [5]:
# Check which standard tile names that could be included in the dataset are missing + which tiles have been annotated in duplicate
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
unlabeled_array = []
duplicated_standard_tile_names = []
for tile_name_standard_complete_array in tile_names_tile_urls_complete_array_unique_standard_tile_names: #iterate over the tiles that are meant to be in full set
    standard_tile_names = [string for string in tile_names_in_storage_space_standard_formatted[:,0] if tile_name_standard_complete_array[2][0:27] in string] #check if a given tile has been annotated
    if len(standard_tile_names) == 0:
        counter_0 += 1
        unlabeled_array.append(tile_name_standard_complete_array)
    if len(standard_tile_names) == 1:
        counter_1 += 1
    if len(standard_tile_names) > 1:
        counter_gr1 += 1
        duplicated_standard_tile_names.append(tile_name_standard_complete_array)
print(counter_0, counter_1, counter_gr1)
unlabeled_array = np.array(unlabeled_array)

for tile in tile_names_to_download:

    ### download the tiles if they are not in the tiles folder 
    tile_name = tile.split("_",4)[4] #+ ".tif" #get the name of the tile that is to be downloaded
    #check if the tile name is contained in the string of complete arrays
    tile_name = [string for string in tile_names_tile_urls_complete_array[:,0] if tile_name in string]          
    if len(tile_name) == 1: #A single tile name # get tile url from the first (only) entry
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1] 
    elif len(np.unique(tile_name)) > 1: # Multiple (different tiles) possibly the same tiles in different states, possible different years
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[0]][0][1]# get tile url
    elif (len(tile_name) > 1): #Multiple different tile names that are the same, probably different naip storage locations
        # get tile url from the second entry 
        tile_url = tile_names_tile_urls_complete_array[tile_names_tile_urls_complete_array[:,0]==tile_name[1]][1][1]
        
# Check whether unlabeled tiles are known to contain errors 
counter_0 = 0
counter_geq1 = 0
unlabeled_array_no_known_errors = []

for tile_name in unlabeled_array: #iterate over the tiles that are meant to be in full set
    standard_tile_names_included_in_error_array = [string for string in tiles_errors[:,0] if tile_name[2][0:27] in string] #check if a given tile has been annotated
    if len(standard_tile_names_included_in_error_array) == 0:
        counter_0 += 1
        unlabeled_array_no_known_errors.append(tile_name)
        
    if len(standard_tile_names_included_in_error_array) >= 1:
        counter_geq1 += 1
print(counter_0, counter_geq1)

unlabeled_array_no_known_errors = np.array(unlabeled_array_no_known_errors)
np.save("unlabeled_tile_names_tile_urls.npy", unlabeled_array_no_known_errors[:,[0,1]])

48 2345 46


In [ ]:
"""
# Determine which tiles included in the complete dataset array have not been labeled (tile names formatted)
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_names_formatted in tile_names_in_storage_space_standard_formatted[:,1]: #iterate over the tiles that have actually been annotated
    strings_with_substring = [string for string in tile_names_tile_urls_complete_array_unique_formatted_tile_names[:,3] if tile_names_formatted in string] #check if a given tile is in the complete_array 
    if len(strings_with_substring) == 0:
        counter_0 += 1
    if len(strings_with_substring) == 1:
        counter_1 += 1
    if len(strings_with_substring) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)

# Check which formatted tile names that could be included in the dataset are missing
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_name_formatted_complete_array in tile_names_tile_urls_complete_array_unique_formatted_tile_names: #iterate over the tiles that are meant to be in full set
    unlabeled = [string for string in tile_names_in_storage_space_standard_formatted[:,1] if tile_name_formatted_complete_array[3][0:46] in string] #check if a given tile has been annotated
    if len(unlabeled) == 0:
        counter_0 += 1
    if len(unlabeled) == 1:
        counter_1 += 1
    if len(unlabeled) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)
"""
# Determine which tiles included in the complete dataset array have not been labeled (standard naming convention)
## Use this to determine which tiles have not been annotated

# Determine the pixel location of the image within the tile

In [ ]:
counter_correct_name = 0
counter_incorrect_name = 0
for jpg in jpgs_without_ext:
    jpg_name_in_image_characteristics = [string for string in image_characteristics['six_digit_chip_name'] if jpg in string] #check if a given tile is in the complete_array 
    if len(jpg_name_in_image_characteristics) >= 1:
        counter_correct_name += 1
    else:
        counter_incorrect_name += 1

In [ ]:
#Identify folders containing positive images

In [7]:
fc.unique_positive_jpgs_from_parent_directory(parent_directory)

(23441, 2)


array([['al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000126.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000126.jpg'],
       ['al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000444.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008808_sw_16_060_20191115_000444.jpg'],
       ['al_60cm_2019_30088_m_3008816_se_16_060_20191115_000051.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\al_60cm_2019_30088_m_3008816_se_16_060_20191115_000051.jpg'],
       ...,
       ['wy_60cm_2019_44107_m_4410749_se_13_060_20190828_000460.jpg',
        '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset\\chips_positive\\wy_60cm_2019_44107_m_4410749_se_13_060_20190828_000460.jpg'],
       ['wy_60cm_2019_44108_m_4410801_ne_12_060_20190821_000041.jpg',
        '/